# Games and Adversarial Search

This notebook serves as supporting material for topics covered in **Chapter 5 - Adversarial Search** in the book *Artificial Intelligence: A Modern Approach.* This notebook uses implementations from the module [games.py](https://github.com/aimacode/aima-python/blob/master/games.py). 

In [2]:
import games
from notebook import psource, pseudocode

ImportError: cannot import name 'psource'

## Representing a game

To represent games we make use of the `Game` class, which we can subclass and override its functions to represent our own games. A helper tool is the namedtuple `GameState`, which in some cases can come in handy, especially when our game needs us to remember a board (like chess).

## `Game` 

Let's have a look at the class `Game`. 
Similarly to the class `Problem`, this class is just a template; we are supposed to create the class for our game, by inheriting from this `Game` class and implementing all the method stubs in `Game`.

In [ ]:
%psource games.Game

Now let's get into details of all the methods in our `Game` class. You have to implement these methods when you create new classes that would represent your game.

* `actions(self, state)`: Given a game state, this method generates all the legal actions possible from this state, as a list or a generator. Returning a generator rather than a list has the advantage that it saves space and you can still operate on it as a list.


* `result(self, state, move)`: Given a game state and a move, this method returns the game state that you get by making that move on this game state.


* `utility(self, state, player)`: Given a terminal game state and a player, this method returns the utility for that player in the given terminal game state. While implementing this method assume that the game state is a terminal game state. The logic in this module is such that this method will be called only on terminal game states.


* `terminal_test(self, state)`: Given a game state, this method should return `True` if this game state is a terminal state, and `False` otherwise.


* `to_move(self, state)`: Given a game state, this method returns the player who is to play next. This information is typically stored in the game state, so all this method does is extract this information and return it.


* `display(self, state)`: This method prints/displays the current state of the game.

## Tic-Tac-Toe

Let's take a look at the class `TicTacToe`. All the methods mentioned in the class `Game` have been implemented here.

In [ ]:
%psource games.TicTacToe

The class `TicTacToe` has been inherited from the class `Game`. As mentioned earlier, you really want to do this. Catching bugs and errors becomes a whole lot easier.

Additional methods in TicTacToe:

* `__init__(self, h=3, v=3, k=3)` :  When you create a class inherited from the `Game` class (class `TicTacToe` in our case), you'll have to create an object of this inherited class to initialize the game. This initialization might require some additional information which would be passed to `__init__` as variables. For the case of our `TicTacToe` game, this additional information would be the number of rows `h`, number of columns `v` and how many consecutive X's or O's are needed in a row, column or diagonal for a win `k`. Also, the initial game state has to be defined here in `__init__`.


* `compute_utility(self, board, move, player)` : A method to calculate the utility of TicTacToe game. If 'X' wins with this move, this method returns 1; if 'O' wins return -1; else return 0.


* `k_in_row(self, board, move, player, delta_x_y)` : This method returns `True` if there is a line formed on TicTacToe board with the latest move else `False.`

### TicTacToe GameState

Now, before we start implementing our `TicTacToe` game, we need to decide how we will be representing our game state. Typically, a game state will give you all the current information about the game at any point in time. When you are given a game state, you should be able to tell whose turn it is next, how the game will look like on a real-life board (if it has one) etc. A game state need not include the history of the game. If you can play the game further given a game state, you game state representation is acceptable. While we might like to include all kinds of information in our game state, we wouldn't want to put too much information into it. Modifying this game state to generate a new one would be a real pain then.

Now, as for our `TicTacToe` game state, would storing only the positions of all the X's and O's be sufficient to represent all the game information at that point in time? Well, does it tell us whose turn it is next? Looking at the 'X's and O's on the board and counting them should tell us that. But that would mean extra computing. To avoid this, we will also store whose move it is next in the game state.

Think about what we've done here. We have reduced extra computation by storing additional information in a game state. Now, this information might not be absolutely essential to tell us about the state of the game, but it does save us additional computation time. We'll do more of this later on.

To store game states will will use the class `GameState`.
`GameState` is a [namedtuple](https://docs.python.org/3.7/library/collections.html#collections.namedtuple).

This class is defined as follows:

`GameState = namedtuple('GameState', 'to_move, utility, board, moves')`

* `to_move`: represents whose turn it is to move next.

* `utility`: stores the utility of the game state. Storing this utility is a good idea, because, when you do a Minimax Search or an Alphabeta Search, you generate many recursive calls, which travel all the way down to the terminal states. When these recursive calls go back up to the original callee, we have calculated utilities for many game states. We store these utilities in their respective `GameState`s to avoid calculating them all over again.

* `board`: A dictionary that stores the board of the game.

* `moves`: stores the list of legal moves possible from the current position.

* `to_move`: A string of a single character, either 'X' or 'O'.

* `utility`: 1 for win, -1 for loss, 0 otherwise.

* `board`: All the positions of X's and O's on the board.

* `moves`: All the possible moves from the current state. Note here, that storing the moves as a list, as it is done here, increases the space complexity of Minimax Search from `O(m)` to `O(bm)`. Refer to Sec. 5.2.1 of the book.

### Representing a move in TicTacToe game

Now that we have decided how our game state will be represented, it's time to decide how our move will be represented.

For our `TicTacToe` game, we'll just represent a move by a tuple, where the first and the second elements of the tuple will represent the row and column, respectively, where the next move is to be made. Whether to make an 'X' or an 'O' will be decided by the `to_move` in the `GameState` namedtuple.

## The minimax algorithm

The *Minimax* algorithm computes the next move for a player (MIN or MAX) at their current state. It recursively computes the minimax value of successor states, until it reaches terminals (the leaves of the tree). Using the `utility` value of the terminal states, it computes the values of parent states until it reaches the initial node (the root of the tree).

It is worth noting that the algorithm works in a depth-first manner.

## Implementation

In the implementation we are using two functions, `max_value` and `min_value` to calculate the best move for MAX and MIN respectively. These functions interact in an alternating recursion; one calls the other until a terminal state is reached. When the recursion halts, we are left with scores for each move. We return the max. Despite returning the max, it will work for MIN too since for MIN the values are their negative (hence the order of values is reversed, so the higher the better for MIN too).

In [ ]:
psource(games.minimax_decision)

## Visualization

Below we have a simple game visualization using the algorithm. After you run the command, click on the cell to move the game along. You can input your own values via a list of 27 integers.

In [ ]:
from notebook import Canvas_minimax
from random import randint

In [ ]:
minimax_viz = Canvas_minimax('minimax_viz', [randint(1, 50) for i in range(27)])

# Alpha-beta pruning

## Overview

While *Minimax* is great for computing a move, it can get tricky when the number of game states gets bigger. The algorithm needs to search all the leaves of the tree, which increase exponentially to its depth.

For Tic-Tac-Toe, where the depth of the tree is 9 (after the 9th move, the game ends), we can have at most 9! terminal states (at most because not all terminal nodes are at the last level of the tree; some are higher up because the game ended before the 9th move). This isn't so bad, but for more complex problems like chess, we have over $10^{40}$ terminal nodes. Unfortunately nobody has found a way to cut the exponent away, but the amount of work can be reduced.

Here we examine *pruning* the game tree, which means removing parts of it that we do not need to examine. The particular type of pruning is called *alpha-beta*, and the search in whole is called *alpha-beta search*.



## Implementation

Like *minimax*, we again make use of functions `max_value` and `min_value`, but this time we utilise the *a* and *b* values, updating them and stopping the recursive call if we end up on nodes with values worse than *a* and *b* (for MAX and MIN). The algorithm finds the maximum value and returns the move that results in it.

The implementation:

In [ ]:
%psource games.alphabeta_search

## Visualization

Below you will find the visualization of the alpha-beta algorithm for a simple game. Click on the cell after you run the command to move the game along. You can input your own values via a list of 27 integers.

In [ ]:
from notebook import Canvas_alphabeta
from random import randint

In [ ]:
alphabeta_viz = Canvas_alphabeta('alphabeta_viz', [randint(1, 50) for i in range(27)])

# PLAYERS

We need more to create an AI that can actually play games. This is where `random_player` and `alphabeta_player` come in.

## query_player
The `query_player` function allows you, a human opponent, to play the game. This function requires a `display` method to be implemented in your game class, so that successive game states can be displayed on the terminal, making it easier for you to visualize the game and play accordingly.

## random_player
The `random_player` is a function that plays random moves in the game. That's it. There isn't much more to this guy. 

## alphabeta_player
The `alphabeta_player`, on the other hand, calls the `alphabeta_search` function, which returns the best move in the current game state. Thus, the `alphabeta_player` always plays the best move given a game state, assuming that the game tree is small enough to search entirely.

## play_game
The `play_game` function will be the one that will actually be used to play the game. You pass as arguments to it an instance of the game you want to play and the players you want in this game. Use it to play AI vs AI, AI vs human, or even human vs human matches!

# LET'S PLAY !


## TicTacToe

Now let's play `TicTacToe`. First we initialize the game by creating an instance of the subclass TicTacToe inherited from the class Game:

In [ ]:
ttt = games.TicTacToe()

We can print a state using the display method:

In [ ]:
ttt.display(ttt.initial)

Let us create a new game state by ourselves to experiment:

In [ ]:
my_state = games.GameState(
    to_move = 'X',
    utility = '0',
    board = {(1,1): 'X', (1,2): 'O', (1,3): 'X',
             (2,1): 'O',             (2,3): 'O',
             (3,1): 'X',
            },
    moves = [(2,2), (3,2), (3,3)]
    )

So, how does this game state look like?

In [ ]:
ttt.display(my_state)

The `random_player` will behave how he is supposed to i.e. *pseudo-randomly*:

In [ ]:
games.random_player(ttt, my_state)

In [ ]:
games.random_player(ttt, my_state)

But the `alphabeta_player` will always give the best move, as expected:

In [ ]:
games.alphabeta_player(ttt, my_state)

Now let's make two players play against each other. We use the `play_game` function for this. The `play_game` function makes players play the match against each other and returns the utility for the first player, of the terminal state reached when the game ends. Hence, for our `TicTacToe` game, if we get the output +1, the first player wins, -1 if the second player wins, and 0 if the match ends in a draw.

In [ ]:
ttt.play_game(games.random_player, games.alphabeta_player)

The output is (usually) -1, because `random_player` loses to `alphabeta_player`. Sometimes, however, `random_player` manages to draw with `alphabeta_player`.

Since an `alphabeta_player` plays perfectly, a match between two `alphabeta_player`s should always end in a draw. Let's see if this happens:

In [ ]:
for _ in range(10):
    print(ttt.play_game(games.alphabeta_player, games.alphabeta_player))

A `random_player` should never win against an `alphabeta_player`. Let's test that.

In [ ]:
for _ in range(10):
    print(ttt.play_game(random_player, alphabeta_player))

## Canvas_TicTacToe(Canvas)

This subclass is used to play TicTacToe game interactively in Jupyter notebooks. TicTacToe class is called while initializing this subclass.

Let's have a match between `random_player` and `alphabeta_player`. Click on the board to call players to make a move.

In [ ]:
from notebook import Canvas_TicTacToe

In [ ]:
bot_play = Canvas_TicTacToe('bot_play', 'random', 'alphabeta')

Now, let's play a game ourselves against a `random_player`:

In [ ]:
rand_play = Canvas_TicTacToe('rand_play', 'human', 'random')

Yay! We (usually) win. But we cannot win against an `alphabeta_player`, however hard we try.

In [ ]:
ab_play = Canvas_TicTacToe('ab_play', 'human', 'alphabeta')